In [1]:
!pip install grpcio==1.42.0 tensorflow-serving-api==2.7.0
!pip install keras-image-helper

In [2]:
import grpc
import tensorflow as tf
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [3]:
host = 'localhost:8500'
channel = grpc.insecure_channel(host)
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [4]:
from keras_image_helper import create_preprocessor

preprocessor = create_preprocessor('xception', target_size=(299, 299))
url = 'https://raw.githubusercontent.com/darkcorpd/ml-zoomcamp/main/kitchenware-classification/cup.jpg'
X = preprocessor.from_url(url)

In [7]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

pb_request = predict_pb2.PredictRequest()
pb_request.model_spec.name = 'tlite_model'
pb_request.model_spec.signature_name = 'serving_default'
pb_request.inputs['input_4'].CopyFrom(np_to_protobuf(X))

In [10]:
pb_response = stub.Predict(pb_request, timeout=20.0)

preds = pb_response.outputs['dense_3'].float_val

In [11]:
classes = [
    'cup',
    'fork',
    'glass',
    'knife',
    'plate',
    'spoon'
]
dict(zip(classes, preds))

{'cup': 3.3057055473327637,
 'fork': -2.321774959564209,
 'glass': -0.3201492130756378,
 'knife': -0.4720998704433441,
 'plate': -0.5078102946281433,
 'spoon': -2.9344308376312256}